<center><h1 style:"text-align:center;">Formulación 1</h1></center>
<center><h3>Relé 1</h3></center>

### 1. Imports

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Acceder al sistema operativo
import os
import glob
import shutil
import random

c:\Users\aherrada\Documents\Git\Pytorch\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Model Parameters

In [2]:
# LSTM parameters
hidden_dim = 20
n_signals = 1
N = 64

# _batch_size => m in figure 1.
train_batch_size = 64
dev_batch_size = 16
test_batch_size = 16

# Classification type (binary)
tagset_size = 1

# Set
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


### 3. Model Creation

##### 3.1. FaultDetector Class Creation

In [3]:
# Model Creation


class FaultDetector(nn.Module):
    """Information about FaultDetector"""

    def __init__(self, N, hidden_dim, tagset_size):
        super(FaultDetector, self).__init__()
        self.lstm = nn.LSTM(N, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, tagset_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        fc_layer = self.fc(lstm_out[:, -1, :])

        return torch.sigmoid(fc_layer)


##### 3.2. Model Instance Initialization

In [4]:
model = FaultDetector(n_signals, hidden_dim, tagset_size).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Learning rate decay (optional)
decayRate = 0.96
my_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(
    optimizer=optimizer, gamma=decayRate
)

print(f"Model structure: {model}\n")

# Number of parameters
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"Number of parameters: {params}")

Model structure: FaultDetector(
  (lstm): LSTM(1, 20, batch_first=True)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

Number of parameters: 1861


### 4. Dataset Creation

##### 4.1. Dataset Class Creation

In [5]:
from utils.signalload import CSV_pandas_path
from utils.auxfunctions import moving_window
import numpy as np
from itertools import repeat


class Form1Dataset(torch.utils.data.Dataset):
    """Some Information about Form1Dataset"""

    def __init__(
        self,
        dataset_dir,
        signal_name,
        max_window_idx=193,
        window_length=64,
        step=1,
        test=False,
    ):
        super(Form1Dataset, self).__init__()
        self.max_window_idx = max_window_idx
        self.test = test
        self.signal_name = signal_name

        self.csv_list = glob.glob(f"{dataset_dir}*.csv")
        self.csv_amount = len(self.csv_list)
        self.windows_amount = max_window_idx * self.csv_amount
        self.window_length = window_length
        self.step = step

    def __getitem__(self, index):

        # sample_settings
        window_length = self.window_length
        step = self.step

        # Calculate window_idx and file name
        window_idx = index % self.max_window_idx
        csv_idx = index // self.max_window_idx
        csv_full_path = self.csv_list[csv_idx]

        # Load CSV, signal and create windows
        csv_name = os.path.basename(csv_full_path)
        signal, t, _ = CSV_pandas_path(csv_full_path).load_data(self.signal_name)
        signal_windows, t_windows = list(
            map(moving_window, [signal, t], repeat(window_length), repeat(step))
        )

        # select window

        signal_window = signal_windows[window_idx]
        signal_window = np.expand_dims(signal_window, axis=1)
        t_window = t_windows[window_idx]

        # create labels
        if window_idx < 66:
            label = np.array([0])
        else:
            label = np.array([1])

        # For dataset visualization return following information
        signal_window = torch.from_numpy(np.copy(signal_window)).float()
        label = torch.from_numpy(label).float()
        if self.test == True:
            return signal_window, t_window, label, csv_name, index, window_idx

        # For training only signals and labels are required

        return signal_window, label

    # def get_events(self, csv_index):
    #     index_min = csv_index * self.max_window_idx
    #     index_max = ((csv_index + 1) * self.max_window_idx) - 1
    #     windows = np.array([])
    #     labels = np.array([])
    #     for i in range(index_min, index_max + 1):
    #         signal_window, label = self.__getitem__(i)
    #         windows = np.append(windows, signal_window, axis=0)
    #         labels = np.append(labels, label)
    #     print(windows.shape)
    #     print(labels.shape)

    def get_event(self, csv_idx):

        # Get indices 
        idx_min = csv_idx * self.max_window_idx
        idx_max = ((csv_idx + 1) * self.max_window_idx) - 1

        # Get path of csv_index
        csv_full_path = self.csv_list[csv_idx]

        # Load CSV, signal and create windows
        csv_name = os.path.basename(csv_full_path)

        # Load CSV, signal and create windows
        csv_name = os.path.basename(csv_full_path)
        signal, t, _ = CSV_pandas_path(csv_full_path).load_data(self.signal_name)
        return signal, t, idx_min, idx_max, csv_name

    def len_events(self):
        return self.csv_amount

    def __len__(self):
        return self.csv_amount * self.max_window_idx


In [ ]:
signal_name = "I: X0071A-R3A"
# signal_name = "I: X0023A-R1A"

dataset = Form1Dataset(
    dataset_dir, max_window_idx=max_window_idx, signal_name=signal_name
)
signal, t, idx_min, idx_max, csv_name = dataset.get_event(1)
plt.plot(t, signal)
signal = dataset.__getitem__(75)[0]
plt.show()
print(signal.shape)
plt.plot(signal)
# print(idx_min, idx_max)
model.eval()
signal = dataset.__getitem__(75)[0]
signal = torch.unsqueeze(signal, 0).to(device)
pred = model(signal)
print(pred)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
iterator = iter(dataloader)
for i in range(0, 76):
    signal, t = next(iterator)
plt.plot(signal[0,:,0])
pred = model(signal)
print(pred)


preds = torch.empty((0, 1)).float().to(device)

for i in range(idx_min, idx_max + 1):
    # signal = dataset[i][0]
    signal = dataset.__getitem__(75)[0]
    # print(signal)
    signal = torch.unsqueeze(signal, 0).to(device)
    # print(signal.shape)

    pred = model(signal)
    preds = torch.cat((preds, pred))
    
    break
# print(preds.shape)
print(preds)


##### 4.2. Database Directory Creation 

In [6]:
# Creating Dataset Folders

dataset_dir_full = "C:/Users/aherrada/OneDrive - Universidad del Norte/Uninorte/DetectionDataBase/LSTM_form_1/fault_01/"
dataset_dir = "C:/Users/aherrada/OneDrive - Universidad del Norte/Uninorte/DetectionDataBase/LSTM_form_1/R1/"

max_window_idx = 130
n_events = 54


def dataset_creator(source_path, target_path, signal_list=False):
    csv_list = glob.glob(f"{source_path}*.csv")
    if not os.path.isdir(target_path):
        print("creating")
        os.makedirs(target_path)
    else:
        file_list = glob.glob(f"{target_path}*.csv")
        for file in file_list:
            os.remove(file)
    csv_list = random.sample(csv_list, n_events)
    print(f"copying {len(csv_list)} files...")
    for csv in csv_list:
        source_path = csv
        target_path = target_path
        shutil.copy(source_path, target_path)
    print("finished")
    
    if signal_list == True:
        print("Available Relays (current):")
        CSV_pandas_path(csv_list[0]).relay_list()


# Crear los 3 Datasets para entrenar los relés individualmente
dataset_creator(dataset_dir_full, dataset_dir, signal_list=False)


copying 54 files...
finished


##### 4.3. Dataset and Dataloader Instances Creation

In [7]:
# Load Dataset
signal_name = "I: X0023A-R1A"
# signal_name = "I: X0004A-R2A"
# signal_name = "I: X0071A-R3A"

dataset = Form1Dataset(
    dataset_dir, max_window_idx=max_window_idx, signal_name=signal_name
)
# dataset.get_events(0)
train_dataset, test_dataset = random_split(dataset, [0.95, 0.05])
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False)
# EL test set realmente es un dev set
# Como regla general, si es utilizado dentro del loop es un dev set.


### 5. Model Training


##### 5.1. Train / Test(dev) Loop Creation

In [8]:
# Create Training / Test / Validation Loops



# Training
def train(dataloader, model, loss_fn, optimizer, return_loss=False):
    size = len(dataloader.dataset)
    model.train()
    loss_list = []

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        # Compute prediction error
        pred = model(x)
        loss = loss_fn(pred, y)
        print(pred)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if return_loss == True:
            loss_list.append(loss.item())
        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>f} [{current:>5d}/{size:>5d}]")

        if batch % 5 == 0:
            my_lr_scheduler.step()
    if return_loss == True:
        return loss_list


def test(dataloader, model, loss_fn, return_metrics=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:

            x, y = x.to(device), y.to(device)
            pred = model(x)
            
            test_loss += loss_fn(pred, y).item()
            correct += (torch.round(pred) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )
    if return_metrics == True:
        return correct * 100, test_loss

from torchmetrics.classification import StatScores
from torchmetrics.classification import BinaryStatScores
from torchmetrics.functional.classification import binary_stat_scores
def test_metrics(dataloader, model, loss_fn, return_metrics=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # metric = BinaryStatScores().to(device)
    conf_matrix = torch.empty((1, 5)).to(device)
    # conf_matrix = conf_matrix.type(torch.int64)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for x, y in dataloader:

            x, y = x.to(device), y.to(device)
            pred = model(x)
            # conf_matrix += metric(torch.round(pred), y)
            conf_matrix += binary_stat_scores(pred, y)
            test_loss += loss_fn(pred, y).item()
            correct += (torch.round(pred) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )
    if return_metrics == True:
        return conf_matrix


##### 5.2. Training the model

In [9]:

epochs = 1

for epoch in range(epochs):
    
    print(f"Epoch {epoch+1}\n--------------------------------")
    # train_loss = train(train_dataloader, model, loss_fn, optimizer, return_loss=True)
    # accuracy, _ = test(test_dataloader, model, loss_fn, return_metrics=True)
    conf_matrix = test_metrics(test_dataloader, model, loss_fn, return_metrics=True)

Epoch 1
--------------------------------


AttributeError: 'function' object has no attribute 'to'

In [10]:
print(conf_matrix.type())
def metrics(conf_matrix):
    print(conf_matrix)

    conf_matrix = conf_matrix.type(torch.int64)
    print(conf_matrix)
    TP = int(conf_matrix[0, 0].item())
    FP = int(conf_matrix[0, 1].item())
    TN = int(conf_matrix[0, 2].item())
    FN = int(conf_matrix[0, 3].item())
    P = TP + FN
    N = TN + FP
    TOTAL = TP + FP + TN + FN
    TPR = TP / (TP + FN)
    FPR = FP / (FP + TN)
    TNR = TN / (FP + TN)
    ACC = (TP + TN) / (TOTAL)
    
    print(TPR)
    print(FPR)
    print(ACC)

metrics(conf_matrix)


NameError: name 'conf_matrix' is not defined

##### 5.3. Loss visualization

In [ ]:
mini_batch_list = [index for index, _ in enumerate(train_loss)]
train_loss_ewm = pd.DataFrame(train_loss).ewm(com=0.95).mean()
plt.plot(mini_batch_list, train_loss, mini_batch_list, train_loss_ewm)

##### 5.4. Saving the Model

In [ ]:
# # Save the Model
# torch.save(model.state_dict(), './models/LSTMHarmonic_weights_989.pth')
# model.load_state_dict(torch.load('./models/LSTMHarmonic_weights_R3_934.pth'))
model.load_state_dict(torch.load('./models/LSTMHarmonic_weights_R1_996.pth'))
# model.load_state_dict(torch.load('./models/LSTMHarmonic_weights_R2_978.pth'))

# R2 + R3 weights

### 6. Results Visualization


#### 6.1. Confussion Matrix Results Table Function

In [ ]:
# Data visualization (CPU)


def confussion_matrix_labels(pred_label, true_label):
    label = ""
    if int(pred_label) == int(true_label):
        label += "True"
    else:
        label += "False"
    if pred_label == 1:
        label += " positive"
    else:
        label += " negative"
    return label


def confussion_matrix(x, labels):
    data = {
        "Pred probability": torch.reshape(x, (-1,)).cpu().numpy(),
        "Pred label": torch.reshape(torch.round(x), (-1,)).int().cpu().numpy(),
        "True label": torch.reshape(labels, (-1,)).int().cpu().numpy(),
    }
    df = pd.DataFrame(data)
    df["prueba"] = confussion_matrix_pandas(df["Pred label"], df["True label"])
    return df


confussion_matrix_pandas = np.vectorize(confussion_matrix_labels)


##### 6.2. Plotly Plot

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fs = 3840
dataset_plot = Form1Dataset(
    dataset_dir, max_window_idx=max_window_idx, signal_name=signal_name, test=True
)
plot_dataloader = DataLoader(dataset_plot, batch_size=train_batch_size, shuffle=True)

dataiter = iter(plot_dataloader)
signal_windows, t_windows, labels, csv_name, index, window_idx = next(dataiter)
print(len(csv_name))
print(window_idx.shape)
print(labels.shape)
x = model(signal_windows[0:9].to(device))
x = x.detach()

# Crear Matriz de confusión
df = confussion_matrix(x, labels[0:9])
print(df)

# figure = plt.figure(figsize=(10, 10))
cols, rows = 3, 3
fig = make_subplots(rows=rows, cols=cols)

label_to_text = {
    "0": "No Fault",
    "1": "Fault",
}

for i in range(0, cols * rows):

    col = i % cols
    row = i // cols
    label = label_to_text[str(int(labels[i, 0].item()))]
    win_idx = str(window_idx[i].item())
    fig.add_trace(
        go.Scatter(
            name=f"{label} - {win_idx} - {csv_name[i]}",
            x=t_windows[i, :],
            y=signal_windows[i, :, 0],
            marker_color="black",
        ),
        row=row + 1,
        col=col + 1,
    )

fig.update_layout(
    height=700,
    width=900,
    title_text="Figuras",
    margin={"b": 20, "t": 40, "l": 20, "r": 30},
    plot_bgcolor="rgba(0,0,0,0)",
    # paper_bgcolor="whitesmoke",
)
fig.show()


##### 6.3. Matlab Plots

In [ ]:
dataset_plot = Form1Dataset(
    dataset_dir, max_window_idx=max_window_idx, signal_name=signal_name, test=True
)
plot_dataloader = DataLoader(dataset_plot, batch_size=train_batch_size, shuffle=True)

dataiter = iter(plot_dataloader)
signal_windows, t_windows, labels, *_ = next(dataiter)
fs = 3840

x = model(signal_windows[1:10].to(device))
x = x.detach()

# Crear Matriz de confusión
df = confussion_matrix(x, labels[1:10])
print(df)

figure = plt.figure(figsize=(10, 10))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    figure.add_subplot(rows, cols, i)
    plt.plot(t_windows[i, :], signal_windows[i, :, 0])


### 7. References
1. PyTorch Contributors: LSTM. [LSTM — PyTorch 1.13 documentation](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html).
<!-- @online{LSTM,
    author = "{PyTorch Contributors}",
    title = "LSTM",
    url  = "https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html",
    addendum = "(accessed: 10.02.2023)",
    keywords = "LSTM pytorch"
} -->